In [134]:
import sympy as sp
import numpy as np
import time

In [137]:
f = sp.Function('f')
variables = 10
x = sp.Matrix([0]*variables)
for i in range(variables):
    exec(f"x{i} = sp.symbols('x{i}')")
    x[i] = eval(f"x{i}")
#f = (x0**2 + x1**2)+sp.sin(x0)+sp.cos(x1)
#f = (1-x0)**2 + 100*(x1-x0**2)**2
f = (x0+2)**2 + (x1+2)**2 + (x2+5)**2 + (x3+2)**2 + (x4+4)**2 + (x5-5)**2 + (x6+1)**2 + (x7)**2 + (x8+2)**2 + (x9-1)**2
display(f)
fd = f.diff(x)
display(fd)
# Lambda function
f_lambda = sp.lambdify(x, f, 'numpy')
fd_lambda = sp.lambdify(x, fd, 'numpy')
# Plot the function
sp.plotting.plot3d(f, (x0, -5, 5), (x1, -5, 5))

x7**2 + (x0 + 2)**2 + (x1 + 2)**2 + (x2 + 5)**2 + (x3 + 2)**2 + (x4 + 4)**2 + (x5 - 5)**2 + (x6 + 1)**2 + (x8 + 2)**2 + (x9 - 1)**2

Matrix([
[ 2*x0 + 4],
[ 2*x1 + 4],
[2*x2 + 10],
[ 2*x3 + 4],
[ 2*x4 + 8],
[2*x5 - 10],
[ 2*x6 + 2],
[     2*x7],
[ 2*x8 + 4],
[ 2*x9 - 2]])

ValueError: The number of free_symbols in the expression is greater than 2

In [147]:
def gradientDecent(x, min_value=-np.inf, max_value=np.inf, tol=1e-3, max_iter=20, momentum=0.01, learning_rate=1):
    x = np.array(x, dtype=float)
    velocity = np.zeros_like(x)
    previous_tol = 1 
    stale = 0
    for i in range(max_iter):
        gradient = -fd_lambda(*x)
        gradient = gradient.reshape(-1)        
        # Update velocity
        velocity = momentum * velocity + learning_rate * gradient
        # Line search
        cost = f_lambda(*x)
        newCost = f_lambda(*(x+velocity))
        a = 1
        for j in range(10):
            if newCost > cost * 0.95/a:
                a *= 0.8
                newCost = f_lambda(*(x+velocity*a))
                print(f'Line search, a = {np.round(a, 2)}, cost = {np.round(cost, 2)}, newCost = {np.round(newCost, 2)}')
            else:
                velocity *= a
                break
        if newCost >= cost:
            stale += 1
        learning_rate *= 0.8
        # Update x
        x += velocity
        # Clamp x to min and max values
        x = np.clip(x, min_value, max_value)               
        # Check if converged
        current_tol = np.linalg.norm(gradient)
        print(f'Iteration {i+1}, x = {np.round(x, 2)}, gradient = {np.round(np.linalg.norm(gradient), 2)}, velocity = {np.round(np.linalg.norm(velocity), 2)}, tol = {current_tol}')     
        if round(previous_tol, 7) == round(current_tol, 7):
            stale += 1
            if stale == 2:
                print(f'Converged in {i+1} iterations (Reached a plateau)')
                return x
        previous_tol = current_tol
        if np.linalg.norm(gradient) < tol:
            print(f'Converged in {i+1} iterations')
            return x
    return x

# design variables, lagrange multipliers, and slack variables
x = [*[-100]*variables]
constraints = [-10, 10]
start = time.time()
result = gradientDecent(x, *constraints)
print(f'Elapsed time: {time.time()-start}')

Line search, a = 0.8, cost = 97684.0, newCost = 35166.24
Iteration 1, x = [10. 10. 10. 10. 10. 10. 10. 10. 10. 10.], gradient = 625.09, velocity = 500.07, tol = 625.0887936925442
Iteration 2, x = [ -7.63  -7.63 -10.    -7.63 -10.     3.68  -6.02  -4.4   -7.63  -2.78], gradient = 72.77, velocity = 53.4, tol = 72.77362159464101
Iteration 3, x = [-0.6  -0.6  -3.82 -0.6  -2.53  5.31  0.24  1.09 -0.6   1.93], gradient = 31.53, velocity = 19.65, tol = 31.525165059044504
Iteration 4, x = [-1.96 -1.96 -4.97 -1.96 -3.96  5.01 -0.97  0.03 -1.96  1.02], gradient = 7.77, velocity = 3.78, tol = 7.768436077564141
Iteration 5, x = [-2.01 -2.01 -5.01 -2.01 -4.01  5.   -1.01 -0.01 -2.01  1.  ], gradient = 0.21, velocity = 0.12, tol = 0.206517667531556
Iteration 6, x = [-2. -2. -5. -2. -4.  5. -1. -0. -2.  1.], gradient = 0.04, velocity = 0.01, tol = 0.038307854542657
Iteration 7, x = [-2. -2. -5. -2. -4.  5. -1. -0. -2.  1.], gradient = 0.02, velocity = 0.0, tol = 0.01564817660434174
Iteration 8, x = [

In [ ]:
# Compute max step size to stay within bounds
        aMax = 1
        for j in range(variables):
            if gradient[j] == 0:
                continue
            a = (min_value-x[j])/gradient[j] if gradient[j] < 0 else (max_value-x[j])/gradient[j]
            aMax = min(aMax, a)
# Line search
        cost = f_lambda(*x)
        newCost = f_lambda(*(x+gradient*a))
        for j in range(10):
            if newCost < cost * 0.99/a:
                break
            a *= 0.8
            newCost = f_lambda(*(x+gradient*a))
            print(f'Line search, a = {np.round(a, 2)}, cost = {np.round(cost, 2)}, newCost = {np.round(newCost, 2)}, aMax = {np.round(aMax, 2)}')

In [ ]:
    

def IPM(x, constraints, max_iter=20, tol=1e-6):
    x = np.array(x, dtype=float)
    mu = 1.0
    tolCurrent = np.inf
    for i in range(max_iter):
        # Newton's method
        step, A, b = newtonSolver(x, constraints, mu)   
        
        # Max step size for design variables
        aMax = 1
        for j in range(constraintsNum):
            a = (0.005-1)*x[variables+constraintsNum+j]/step[variables+constraintsNum+j]
            if a > 0:
                aMax = min(aMax, a)
        aDesign = aMax

        # Line search
        ak = aMax
        cost = f_lambda(*x[:variables])
        newCost = f_lambda(*(x[:variables]+step.reshape(-1)[:variables]*ak))
        while newCost > cost + 1e-4*ak*(fd_lambda(*x[:variables]).T @ step[:variables]):
            ak *= 0.9
            newCost = f_lambda(*(x[:variables]+step.reshape(-1)[:variables]*ak))

        # Max step size for design variables
        aMax = 1
        for j in range(constraintsNum):
            a = (0.005-1)*x[variables+j]/step[variables+j]
            if a > 0:
                aMax = min(aMax, a)
        aLagrange = aMax

        # Update x
        step = step.reshape(-1)
        step[:variables] = step[:variables]*ak
        step[variables:variables+constraintsNum] = step[variables:variables+constraintsNum]*aLagrange
        step[variables+constraintsNum:] = step[variables+constraintsNum:]*ak
        x += step

        # Check convergence
        tolCurrent = np.linalg.norm(fd_lambda(*x[:variables]))
        print(f'Iteration {i+1}, tol = {np.round(tolCurrent, 3)}, mu = {np.round(mu, 2)}, ak = {ak}, al = {aLagrange} ad = {aDesign} \nx = {np.round(x, 2)}')
        if tolCurrent < tol:
            print('Converged')
            return x
        if ak < tol or aLagrange < tol:
            print('Line search failed')
            print(step)
            return x
        # Update mu
        mu *= 0.5
    return x